<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

import re
import urllib.request
import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs


### Datos
Utilizaremos el mismo dataset de "Climate change" utilizado en la práctica de clase 2

In [2]:
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Climate_change')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [3]:
citas = 357 #Numero de citas maximas
for i in range(citas):
  article_text = article_text.replace('['+str(i+1)+']', '')

In [4]:
article_text = article_text.replace('\n', '') #Remove end of line '\n'
article_text = re.split(r"\.\s*", article_text) #Split using period '.'

In [5]:
df = pd.DataFrame(article_text)

In [6]:
df.head()

,0
0,contemporary climate change includes both glob...
1,there have been previous periods of climate ch...
2,"instead, they are caused by the emission of gr..."
3,burning fossil fuels for energy use creates mo...
4,"certain agricultural practices, industrial pro..."


In [7]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 523


### 1 - Preprocesamiento

In [8]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [9]:
# Demos un vistazo
sentence_tokens[:2]

[['contemporary',
  'climate',
  'change',
  'includes',
  'both',
  'global',
  'warming',
  'and',
  'its',
  'impacts',
  'on',
  "earth's",
  'weather',
  'patterns'],
 ['there',
  'have',
  'been',
  'previous',
  'periods',
  'of',
  'climate',
  'change',
  'but',
  'the',
  'current',
  'changes',
  'are',
  'distinctly',
  'more',
  'rapid',
  'and',
  'not',
  'due',
  'to',
  'natural',
  'causes']]

### 2 - Crear los vectores (word2vec)

In [10]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [11]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=3,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [12]:
# Buildear el vocabularui con los tokens
w2v_model.build_vocab(sentence_tokens)

In [13]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 523


In [14]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 310


### 3 - Entrenar el modelo generador

In [15]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 93005.5
Loss after epoch 1: 42993.875
Loss after epoch 2: 41966.671875
Loss after epoch 3: 42387.859375
Loss after epoch 4: 42869.875
Loss after epoch 5: 42536.25
Loss after epoch 6: 43060.46875
Loss after epoch 7: 41887.3125
Loss after epoch 8: 42469.59375
Loss after epoch 9: 43115.0
Loss after epoch 10: 42605.0625
Loss after epoch 11: 43553.40625
Loss after epoch 12: 43214.4375
Loss after epoch 13: 42522.1875
Loss after epoch 14: 42522.4375
Loss after epoch 15: 42746.625
Loss after epoch 16: 42653.1875
Loss after epoch 17: 42943.4375
Loss after epoch 18: 42480.75
Loss after epoch 19: 41608.75
Loss after epoch 20: 41042.5625
Loss after epoch 21: 41224.0625
Loss after epoch 22: 40307.1875
Loss after epoch 23: 39130.375
Loss after epoch 24: 35183.375
Loss after epoch 25: 35223.5
Loss after epoch 26: 35379.75
Loss after epoch 27: 35501.625
Loss after epoch 28: 35604.0
Loss after epoch 29: 34425.875
Loss after epoch 30: 34659.0
Loss after epoch 31: 34137.0
Loss after e

(346606, 859800)

### 4 - Ensayar

In [16]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["heat"], topn=10)

[('scenario', -0.024284332990646362),
 ('many', -0.028013568371534348),
 ('less', -0.035465121269226074),
 ('could', -0.03770184889435768),
 ('limiting', -0.04398142918944359),
 ('emissions', -0.04751690477132797),
 ('major', -0.053143538534641266),
 ('global', -0.05630140379071236),
 ('mitigation', -0.06626509130001068),
 ('health', -0.07756134867668152)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["flooding"], topn=10)

[('coastal', 0.7824815511703491),
 ('extreme', 0.7419816851615906),
 ('increases', 0.6694974899291992),
 ('areas', 0.6064659357070923),
 ('sea', 0.5930067300796509),
 ('melting', 0.5905830264091492),
 ('rise', 0.5903602838516235),
 ('weather', 0.5635567903518677),
 ('space', 0.5601062178611755),
 ('loss', 0.5343263745307922)]

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["economic"], topn=5)

[('rapid', 0.5980287790298462),
 ('food', 0.5834060311317444),
 ('development', 0.5705217123031616),
 ('clean', 0.5523902773857117),
 ('extreme', 0.5489441156387329)]

In [19]:
# Ensayar con una palabra que no está en el corpus (en vocab):
w2v_model.wv.most_similar(negative=["change"])

[('with', -0.06241224333643913),
 ('effect', -0.09461347758769989),
 ('level', -0.09754686802625656),
 ('greater', -0.10055343806743622),
 ('future', -0.11790648847818375),
 ('net', -0.1277923882007599),
 ('vapour', -0.13858191668987274),
 ('air', -0.14576923847198486),
 ('dioxide', -0.1473863124847412),
 ('increases', -0.14773641526699066)]

### 5 - Visualizar agrupación de vectores

In [20]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [21]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=300
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
